<a href="https://colab.research.google.com/github/gtbook/robotics/blob/main/S31_vacuum_state.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%pip install -q -U gtbook

Note: you may need to restart the kernel to use updated packages.


In [1]:
import gtsam 
import gtbook
import gtbook.display
from gtbook.discrete import Variables
VARIABLES = Variables()
def pretty(obj): 
    return gtbook.display.pretty(obj, VARIABLES)

# State Abstractions

> We introduce discrete states as an abstraction of navigationg through space.

**This Section is still in draft mode and was released for adventurous spirits (and TAs) only.**

In [2]:
from gtbook.display import randomImages
from IPython.display import display
display(randomImages(3, 1, "steampunk", 1))

The concept of state is an abstraction, allowing robots to reason about what is happening to them and their environment as they act. This does not necessarily mean that we, as robot programmers, have no deeper knowledge of what is going on: we do know that a vacuuming robot operates in a two dimensional space, typically a single floor in a domestic residence. However, we might have programmed in "actions" that abstract to the notion of moving from room to room, and hence may only be interested in knowing which room the robot is in for the purposes of triggering those actions. Other actions that are useful but will not discuss further in this section might involve vacuuming the current room, terminating when it is done or when the robot is stuck. Rather, we will concentrate below on navigation actions, moving from room to room, knowing that it might be only a small part of the repertoire that a true vacuuming robot needs to be useful.

## A Vacuuming Robot
<figure>
<img src="https://github.com/gtbook/robotics/blob/main/Figures3/S31-House.png?raw=1" id="fig:House" alt="" />
<figcaption id="fig:House" label="fig:House">The floor plan of a a house in which our hypothetical vacuum robot will operate.</figcaption>
</figure>

As an example, let us look at the floor plan in Figure <a href="#fig:Vacuum" data-reference-type="ref" data-reference="fig:House">1</a>, depicting the ground floor of a house with 5 connected rooms. These rooms will constitute the abstracted **state space** for the robot.

As before, we can create a variable and a prior distribution over this variable. In this chapter we will model the actions of a robot overtime. Hence, the prior distribution will only concern or knowledge about the state at the very first time step. In particular we will assume that the robot *always* starts out in the office, where its charging station is located. The following code does this:

In [3]:
rooms = ["Living Room", "Kitchen", "Office", "Hallway", "Dining Room"]
X = VARIABLES.discrete_series("X", [1], rooms)
prior = gtsam.DiscreteDistribution(X[1], "0/0/1/0/0")
pretty(prior)

X1,value
Living Room,0
Kitchen,0
Office,1
Hallway,0
Dining Room,0


We use a discrete distribution to convey our knowledge that the robot *always* starts out in the office. We do this with $100\%$ certainty in this case. This use of probability values to indicate how certain we are about facts in the world is a "Bayesian" view, as we explain below.

## Bayesian vs. Frequentist

<figure>
<img src="https://github.com/gtbook/robotics/blob/main/Figures3/N3-Bayes.png?raw=1" id="fig: Bayesian" style="width:14cm" alt="" />
<figcaption id="fig: Bayesian" label="fig: Bayesian">The Reverend Thomas Bayes gave his name to associating probabilities with the strength of beliefs rather than a frequency of events, even though this seems to have been first introduced by Laplace.</figcaption>
</figure>

We take a Bayesian view of probability, rather than a frequentist one.
This means that we see probabilities as describing our knowledge about
events, rather than tallying up frequencies by which they occur. Think
of the weather-person talking about the probability of rain tomorrow.
Probabilities viewed this way can be used to describe knowledge about
the state of the world, and how actions affect the state of an agent and
the world.

<figure>
<img src="https://github.com/gtbook/robotics/blob/main/Figures3/N3-Frequentist.png?raw=1" id="fig: Frequentist" style="width:9cm" alt="" /><figcaption id="fig: Frequentist" label="fig: Frequentist">The caricature of the frequentist view involves counting many heads and tails.</figcaption>
</figure>

This is to be contrasted with a frequentist view, where probabilities
are used to describe the frequencies of events in a series of repeated
trials. A Bayesian, instead, might qualify their knowledge about an
event that has not even happened yet, let alone multiple times. Of
course, in most cases this belief is based on experience, i.e., lots of
repeated events in the past, and so it can be seen that perhaps these
views are not so different after all.


## GTSAM 101

Most everything above should by now be familiar, but we used a new `Variables` method, `discrete_series`, to define a time series of state variables. The signature of this method is
```python
def discrete_series(self, character: str, indices: Iterable[int],
                    domain: List[str]) -> List[DiscreteKey]:
    """Create several discrete variables with Symbol names.

    Args:
        character (str): a single character.
        indices: (Iterable[int]): a set of integer indices.
        domain (List[str]): names for the different values.

    Returns:
        List[DiscreteKey], i.e., [(gtsam.Key, cardinality)]
    """
```

For example, the following creates a series of 5 state variables:

In [4]:
states = VARIABLES.discrete_series('X', [1, 2, 3], rooms)

In [5]:
print(states)

{1: (6341068275337658369, 5), 2: (6341068275337658370, 5), 3: (6341068275337658371, 5)}


When we print the results, we see that we now get a *list* of DiscreteKeys, i.e., integer tuples. However, the "Keys" now seem to be very large integers. This is because for series of variables we use the `gtsam.Symbol` type, composed of a single character and an integer index:

In [6]:
symbol = gtsam.Symbol('X', 1)
print(symbol.string())

X1


GTSAM internally stores symbols as a 64-bit integer key, with the 8-bit character in the most significant bits, which explains the large integer value:

In [7]:
print(symbol.key())

6341068275337658369


You can see that this corresponds to the first state above. However, as before, pretty printing translates these into a nicer looking strings wherever it matters.